# Praktikum 3 - HNSW

In [ ]:
!pip install hnswlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=2528149 sha256=7b65eeb2fcfc9ff4e3596ee9863706a0577445181f6fd3f61f4f0d68599ca6cc
  Stored in directory: /root/.cache/pip/wheels/ac/39/b3/cbd7f9cbb76501d2d5fbc84956e70d0b94e788aac87bda465e
Successfully built hnswlib


In [ ]:
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ===========================
# 1. Buat data 2D acak
# ===========================
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

# ===========================
# 2. Exact NN (Brute Force)
# ===========================
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

# ===========================
# 3. HNSW
# ===========================
# Inisialisasi index HNSW
p = hnswlib.Index(space='l2', dim=dim)

# Ukuran maksimum elemen yang bisa ditampung
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data
p.add_items(data)

# Set parameter pencarian
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "detik")


=== Exact NN ===
Indices: [[317 448 892 843  38]]
Distances: [[0.01333264 0.01929028 0.0394427  0.03982243 0.04248696]]
Waktu: 0.04941248893737793 detik

=== HNSW ===
Indices: [[317 448 892 843  38]]
Distances: [[0.00017776 0.00037211 0.00155573 0.00158583 0.00180514]]
Waktu: 0.00018215179443359375 detik


Lakukan percobaan pada metric distance yang berbeda. catat hasilnya pada tabel yang anda buat sendiri seperti pada praktikum 1.

In [ ]:
list_n_points = [1000, 1000000]
list_n_dims = [2, 5]
list_metrics_hnsw = ['l2', 'ip', 'cosine']

In [ ]:
import hnswlib
import numpy as np
import time
import pandas as pd

hnsw_results_list = []

print(f"{'Metric':<10} | {'Dimensions':<10} | {'N Points':<10} | {'Build Time (ms)':<15} | {'Search Time (ms)':<15} | {'Indices Found':<30}")
print("-" * 110)


for n_points in list_n_points:
    for n_dims in list_n_dims:
        for metric in list_metrics_hnsw:
            data = np.random.random((n_points, n_dims)).astype(np.float32)
            query = np.random.random((1, n_dims)).astype(np.float32)
            k = 5 # Number of neighbors to search for
            p = hnswlib.Index(space=metric, dim=n_dims)

            ef_construction = 100
            M = 16
            p.init_index(max_elements=n_points, ef_construction=ef_construction, M=M)

            # Add data to the index and measure build time
            start_build = time.time()
            p.add_items(data)
            end_build = time.time()
            build_time_ms = (end_build - start_build) * 1000

            # Set search parameter (ef)
            # This parameter affects search time and accuracy
            ef_search = 50
            p.set_ef(ef_search)

            # Perform the search and measure search time
            start_search = time.time()
            labels, distances = p.knn_query(query, k=k)
            end_search = time.time()
            search_time_ms = (end_search - start_search) * 1000

            # Store the results
            hnsw_results_list.append({
                'N_Points': n_points,
                'Dimensions': n_dims,
                'Metric': metric,
                'Build_Time_ms': build_time_ms,
                'Search_Time_ms': search_time_ms,
                'Indices': labels.tolist(),
                'Distances': distances.tolist()
            })
            print(f"{metric:<10} | {n_dims:<10} | {n_points:<10} | {build_time_ms:<15.4f} | {search_time_ms:<15.4f} | {str(labels.tolist()):<30}")

    if n_points != list_n_points[-1]:
        print("-" * 110)

df_hnsw_results = pd.DataFrame(hnsw_results_list)
display(df_hnsw_results)

Metric     | Dimensions | N Points   | Build Time (ms) | Search Time (ms) | Indices Found                 
--------------------------------------------------------------------------------------------------------------
l2         | 2          | 1000       | 43.2966         | 0.0858          | [[647, 721, 307, 761, 96]]    
ip         | 2          | 1000       | 31.1575         | 0.0608          | [[329, 377, 839, 591, 743]]   
cosine     | 2          | 1000       | 20.7651         | 0.3076          | [[723, 169, 573, 290, 442]]   
l2         | 5          | 1000       | 31.9529         | 0.0677          | [[796, 819, 541, 846, 281]]   
ip         | 5          | 1000       | 44.4169         | 0.0727          | [[475, 384, 435, 781, 125]]   
cosine     | 5          | 1000       | 38.7208         | 0.0603          | [[37, 957, 603, 591, 953]]    
--------------------------------------------------------------------------------------------------------------
l2         | 2          | 1000000  

,N_Points,Dimensions,Metric,Build_Time_ms,Search_Time_ms,Indices,Distances
0,1000,2,l2,43.296576,0.085831,"[[647, 721, 307, 761, 96]]","[[1.4386238035513088e-05, 0.000310671894112601..."
1,1000,2,ip,31.157494,0.060797,"[[329, 377, 839, 591, 743]]","[[0.10732614994049072, 0.11472171545028687, 0...."
2,1000,2,cosine,20.765066,0.307560,"[[723, 169, 573, 290, 442]]","[[0.0, 1.1920928955078125e-07, 2.3841857910156..."
3,1000,5,l2,31.952858,0.067711,"[[796, 819, 541, 846, 281]]","[[0.050249047577381134, 0.05797947943210602, 0..."
4,1000,5,ip,44.416904,0.072718,"[[475, 384, 435, 781, 125]]","[[-0.7026689052581787, -0.6996986865997314, -0..."
5,1000,5,cosine,38.720846,0.060320,"[[37, 957, 603, 591, 953]]","[[0.005133450031280518, 0.006453514099121094, ..."
6,1000000,2,l2,59803.032875,0.118494,"[[877878, 232344, 904226, 946640, 494704]]","[[3.687381422423641e-08, 1.5928330299175286e-0..."
7,1000000,2,ip,42162.757874,0.086069,"[[992232, 913738, 851885, 21924, 234245]]","[[0.8870553970336914, 0.8870813846588135, 0.88..."
8,1000000,2,cosine,98193.661451,0.182867,"[[248693, 251430, 632709, 777337, 846857]]","[[-2.384185791015625e-07, -2.384185791015625e-..."
9,1000000,5,l2,97999.911785,0.138044,"[[46881, 787084, 870574, 926221, 728511]]","[[0.0008221324533224106, 0.0015844672452658415..."
